In [ ]:
import os,cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [ ]:
model = model.load("classifier.h5")

In [ ]:
path = 'data/'
data = []
labels = []
j = 0
for folders in tqdm(os.listdir(path)):
    folder = path + folders
    for images_path in os.listdir(folder):
        j += 1
        img_path = folder + '/' + images_path
        image = cv2.imread(img_path)
        image = cv2.resize(image, (512,512))
        imagex = np.array(img1)
        img2 = np.expand_dims(img1,axis=0)
        img2 = np.array(img2)

        i = model.predict(img2)[0]
        if i < 0.4:
            label = 'Normal'
            cv2.putText(image, text=label,(10, 20),cv2.FONT_HERSHEY_TRIPLEX,3,(0, 255, 0),3)
            cv2.imwrite('output_dir/' + str(j) + '.jpg', image)
        else:
            label = 'Defect'
            grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])

            with tf.GradientTape() as tape:
                conv_outputs, predictions = grad_model(np.array([imagex]))
                loss = predictions[:,0]

            output = conv_outputs[0]
            grads = tape.gradient(loss, conv_outputs)[0]  * 1e+08
            gate_f = tf.cast(output > 0, 'float32')
            gate_r = tf.cast(grads > 0, 'float32')
            guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

            weights = tf.reduce_mean(guided_grads, axis=(0, 1))

            cam = np.ones(output.shape[0: 2], dtype = np.float32)

            for i, w in enumerate(weights):
                cam += w * output[:, :, i]

            cam = cv2.resize(cam.numpy(), (img.shape[1], img.shape[0]))
            denom = (cam.max() - cam.min())
            cam = np.maximum(cam, 0)
            heatmap = (cam - cam.min()) / denom
            cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
            output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 0.5, 0)
            cv2.putText(output_image, text=label,(10, 20),cv2.FONT_HERSHEY_TRIPLEX,3,(0, 255, 0),3)
            cv2.imwrite('output_dir/' + str(j) + '.jpg', output_image)